In [ ]:
try:
    import os
    import numpy as np
    from astropy.io import fits
    from scipy.ndimage import convolve
    from astropy.coordinates import SkyCoord
    from astropy.wcs import WCS
    from astropy.nddata import Cutout2D
    from astropy import units as u
    from photutils.segmentation import SegmentationImage
    from segm_create import *
    import matplotlib.pyplot as plt

    # Set environment variables
    os.environ["OMP_NUM_THREADS"] = "1"
    
    
except ImportError as e:
    print(f"Error: {e}") 
    


In [ ]:
data_path = '/media/himanshu/MyHDD/data_galaxy_evolution/data/'  # path to HST-JWST fits images
# loading EELGS and control galaxy samples
catalog = data_path + 'sample_details/'
eelgs = catalog + 'eelgs_28aug2024.dat'
control = catalog + 'control_28aug2024.dat'

In [ ]:
c0 = fits.open(data_path+'/catalog/hlsp_jades_jwst_nircam_goods-s-deep_photometry_v2.0_catalog.fits')
c1 = fits.open(data_path+'/catalog/hlsp_jades_jwst_nirspec_goods-s-deephst_clear-prism_line-fluxes_v1.0_catalog.fits')
c2 = fits.open(data_path+'/catalog/hlsp_jades_jwst_nirspec_goods-s-deephst_gratings_line-fluxes_v1.0_catalog.fits')
c3 = fits.open(data_path+'/catalog/hlsp_jades_jwst_nirspec_goods-s_gratings-line-fluxes_v1.1_catalog.fits')
c4 = fits.open(data_path+'/catalog/hlsp_jades_jwst_nirspec_goods-s_prism-line-fluxes_v1.1_catalog.fits')


In [ ]:
# extracting sample data : id, ra, dec, photo_z, spec_z
use_control = False
if use_control == True:
    data_columns = np.loadtxt(control, dtype='str', delimiter=',')[0]
    sample_data  = np.loadtxt(control, delimiter=',', skiprows=1)
else:
    
    data_columns = np.loadtxt(eelgs, dtype='str', delimiter=',')[0]
    sample_data  = np.loadtxt(eelgs, delimiter=',', skiprows=1)
    
new_dtype = [(name, float) for name in data_columns]

sample_data = np.array([tuple(row) for row in sample_data], dtype=new_dtype)

ind = np.where(sample_data['zspec'] > 3.08)
sample_data = sample_data[ind]

offset = 0.3*u.arcsecond
offset = offset.to(u.deg)
sample_data['dec'] = sample_data['dec'] - offset.value


In [ ]:
def load_fits(filt0, filt1):
    '''
    filt0: name of the filter
    
    filt1: name of the filter
    
    returns: fits filters and weights
    '''
    fitsfile0     = data_path  + 'hlsp_fresco_hst_acs-wfc_goods-s_%s_v1.0_sci.fits'%filt0
    fitsfile_wts0 = data_path  + 'hlsp_fresco_hst_acs-wfc_goods-s_%s_v1.0_wht.fits'%filt0
    fitsfile1     = data_path  + 'hlsp_fresco_hst_wfc3-uvis_goods-s_%s_v1.0_sci.fits'%filt1
    fitsfile_wts1 = data_path  + 'hlsp_fresco_hst_wfc3-uvis_goods-s_%s_v1.0_wht.fits'%filt1
    
    return fitsfile0, fitsfile_wts0, fitsfile1, fitsfile_wts1

In [ ]:
filt0='f814w' # non-ionizing filter
filt1='f336w' # ionizing filter
fitsfile0, fitsfile_wts0, fitsfile1, fitsfile_wts1 = load_fits(filt0='f814w', filt1='f336w')

In [ ]:
#loading the filters
f0  = fits.open(fitsfile0)
fw0 = fits.open(fitsfile_wts0)

f1  = fits.open(fitsfile1)
fw1 = fits.open(fitsfile_wts1)

flux_units = f0[0].header['bunit']
# Check if FILTER2 or FILTER is present in the header
if 'FILTER2' in f0[0].header or 'FILTER' in f[0].header:
    # Access the filters, defaulting to empty string if not present
    filter2 = f0[0].header.get('FILTER2', '')
    filter1 = f0[0].header.get('FILTER', '')

    # Determine the filter that starts with 'F'
    filt = filter2 if filter2.startswith('F') else (filter1 if filter1.startswith('F') else None)
    
    if filt:
        print(f"Selected Filter: {filt}")
    else:
        print("No filter starts with 'F'.")
else:
    print("Neither FILTER2 nor FILTER is present in the header.")


In [ ]:
filters = f0, fw0, f1, fw1

In [ ]:
peak_cutout_filt0_arr, peak_cutout_wts0_arr, peak_cutout_filt1_arr, peak_cutout_wts1_arr, err_ind = get_cutout_data_array_v2(sample_data, filters=filters, size=5, use_control=False)

In [ ]:
# checking shape in each cutout
for i in range(len(peak_cutout_filt0_arr)):
    print(f"Cutout {i}: peak_cutout_filt0_arr shape:", peak_cutout_filt0_arr[i].shape)

In [ ]:
# fixing the shape of bad cutouts
# getting the data from cutouts
img_filt0_arr, img_filt1_arr = [], []
wts_filt0_arr, wts_filt1_arr = [], []

for i in range(len(peak_cutout_filt0_arr)):
    img_filt0_arr.append(peak_cutout_filt0_arr[i].data)
    img_filt1_arr.append(peak_cutout_filt1_arr[i].data)
    wts_filt0_arr.append(peak_cutout_wts0_arr[i].data )
    wts_filt1_arr.append(peak_cutout_wts1_arr[i].data )
    
img_filt0_arr_20 = np.copy(img_filt0_arr[20])
img_filt1_arr_20 = np.copy(img_filt1_arr[20])
wts_filt0_arr_20 = np.copy(wts_filt0_arr[20])
wts_filt1_arr_20 = np.copy(wts_filt1_arr[20])

img_filt0_arr[20] = np.zeros(img_filt0_arr[0].shape)
img_filt1_arr[20] = np.zeros(img_filt0_arr[0].shape)
wts_filt0_arr[20] = np.zeros(img_filt0_arr[0].shape)
wts_filt1_arr[20] = np.zeros(img_filt0_arr[0].shape)

img_filt0_arr[20][:77, :125] = img_filt0_arr_20[25:, :125]
img_filt1_arr[20][:77, :125] = img_filt1_arr_20[25:, :125]
wts_filt0_arr[20][:77, :125] = wts_filt0_arr_20[25:, :125]
wts_filt1_arr[20][:77, :125] = wts_filt1_arr_20[25:, :125]


img_filt0_arr = np.where(img_filt0_arr==0, np.nan, img_filt0_arr) 
img_filt1_arr = np.where(img_filt1_arr==0, np.nan, img_filt1_arr)
wts_filt0_arr = np.where(wts_filt0_arr==0, np.nan, wts_filt0_arr)
wts_filt1_arr = np.where(wts_filt1_arr==0, np.nan, wts_filt1_arr)



In [ ]:
for i in range(len(img_filt0_arr)): #fixing the shape of bad cutouts
    print(f"Cutout {i}: peak_cutout_filt0_arr shape:", img_filt0_arr[i].shape)

In [ ]:
# plotting the cutouts
# Define the number of rows and columns for the grid
rows, cols = 6, 5

# Create the figure and axes for the grid
fig, axes = plt.subplots(rows, cols, figsize=(15, 15))

fig.suptitle(r'$f814$', fontsize=20, fontweight='bold', y=1.01)
# Flatten the axes array to iterate easily
axes = axes.flatten()

# Calculate global min and max for scaling
#all_images = np.array(peak_cutout_wts0_arr)  # Assuming this is the list/array of images


# Plot each image on the grid
for i, ax in enumerate(axes):
    if i < len(img_filt0_arr):
        # Plot the image with consistent scaling
        xpix, ypix = peak_cutout_filt0_arr[i].wcs.world_to_pixel_values(sample_data['ra'][i]*u.deg, sample_data['dec'][i]*u.deg)    
        if i ==20:
            ypix = ypix - 25
        ax.imshow(np.ma.masked_equal(img_filt0_arr[i], value=0), origin='lower', cmap='viridis', vmin=-0.1, vmax=0.1)
        ax.plot(xpix, ypix, color='black', marker='x', markersize=10,)  
        #ax.imshow(mask_arr[i], origin='lower', alpha=0.2)
        ax.set_title(r"$\rm %d$" % i, fontsize=8)

        # Disable ticks
        ax.set_xticks([])  # Disable x-axis ticks
        ax.set_yticks([])  # Disable y-axis ticks
    else:
        # Hide the empty subplots
        ax.axis('off')  # Hide unused slots

# Adjust layout to minimize space between images
plt.tight_layout(pad=-0.1, h_pad=0.01, w_pad=0.01)
#plt.savefig('control_gal_samples_final_f336.jpg', bbox_inches='tight', dpi=220)
# Show the plot
plt.show()

In [ ]:
# plotting the cutouts
# Define the number of rows and columns for the grid
rows, cols = 6, 5

# Create the figure and axes for the grid
fig, axes = plt.subplots(rows, cols, figsize=(15, 15))

fig.suptitle(r'$f336$', fontsize=20, fontweight='bold', y=1.01)
# Flatten the axes array to iterate easily
axes = axes.flatten()

# Calculate global min and max for scaling
#all_images = np.array(peak_cutout_wts0_arr)  # Assuming this is the list/array of images


# Plot each image on the grid
for i, ax in enumerate(axes):
    if i < len(img_filt0_arr):
        # Plot the image with consistent scaling
        xpix, ypix = peak_cutout_filt0_arr[i].wcs.world_to_pixel_values(sample_data['ra'][i]*u.deg, sample_data['dec'][i]*u.deg)    
        if i ==20:
            ypix = ypix - 25
        
        #ax.imshow(np.ma.masked_equal(img_filt0_arr[i], value=0), origin='lower', cmap='Blues', vmin=-0.1, vmax=0.1, alpha=0.5)
        ax.imshow(np.ma.masked_equal(img_filt1_arr[i], value=0), origin='lower', cmap='viridis', vmin=-0.1, vmax=0.1,)
        ax.plot(xpix, ypix, color='black', marker='x', markersize=10)  
        #ax.imshow(mask_arr[i], origin='lower', alpha=0.2)
        ax.set_title(r"$\rm %d$" % i, fontsize=8)

        # Disable ticks
        ax.set_xticks([])  # Disable x-axis ticks
        ax.set_yticks([])  # Disable y-axis ticks
    else:
        # Hide the empty subplots
        ax.axis('off')  # Hide unused slots

# Adjust layout to minimize space between images
plt.tight_layout(pad=-0.1, h_pad=0.01, w_pad=0.01)
#plt.savefig('control_gal_samples_final_f336.jpg', bbox_inches='tight', dpi=220)
# Show the plot
plt.show()

In [ ]:
# checking for good cutouts, this checks the indicies of cutouts that are good, deleting the err_ind (error indices)
good_ind  = np.arange(len(sample_data))
if err_ind.size !=0:
    good_ind  = np.delete(good_ind, err_ind)

In [ ]:
# finding matching galaxies from JADEs catalog
# these are already matched catalog stored in .npy files
tol = 0.0003
if use_control == True:
    cat_indx_match = np.load('../data/cat_matching_index_tol_control_%s.npy'%tol, allow_pickle=True)
else:
    cat_indx_match = np.load('../data/cat_matching_index_tol_eelgs_%s.npy'%tol, allow_pickle=True)
    
# storing the IDs, photo_z, and spec_z of the matched galaxies with 68, 95, 99 percent cofidence intervals
# also deleting the error indices
if len(err_ind) != 0:
    cat_indx_match = np.delete(cat_indx_match, err_ind, axis=0)
catalog_ids = c0[-1].data['ID']
phot_z      = c0[-1].data['EAZY_z_a']
z_68l       = c0[-1].data['EAZY_l68']
z_68u       = c0[-1].data['EAZY_u68']
z_95l       = c0[-1].data['EAZY_l95']
z_95u       = c0[-1].data['EAZY_u95']
z_99l       = c0[-1].data['EAZY_l99']
z_99u       = c0[-1].data['EAZY_u99'] 

# keeping photmeteric redshifts and 99% confidence intervals
phot_zz_arr = []
z99ll_arr = []
for ind in range(len(good_ind)):
    phot_zz = c0[-1].data[cat_indx_match[ind][0]]['EAZY_z_a']
    z99ll = z_99l[cat_indx_match[ind][0]]
    z99ul = z_99u[cat_indx_match[ind][0]]

In [ ]:
segm_deblend_arr_f0, segment_arr_f0, data_sub_arr_f0, convolved_data_arr_f0,\
    closest_mask_arr_f0, background_filled_arr2_f0, background_filled_arr3_f0, err_ind_f0 = get_segments_background(img_filt0_arr, box_size_=8,\
                                                                                                                                filt_size=7,\
                                                                                                                                npixels=20,\
                                                                                                                                nlevels=32,\
                                                                                                                                contrast=0.001)

In [ ]:
segm_deblend_arr_f1, segment_arr_f1, data_sub_arr_f1, convolved_data_arr_f1,\
    closest_mask_arr_f1, background_filled_arr2_f1, background_filled_arr3_f1, err_ind_f1 = get_segments_background(img_filt1_arr, box_size_=8,\
                                                                                                                                filt_size=7,\
                                                                                                                                npixels=20,\
                                                                                                                                nlevels=32,\
                                                                                                                                contrast=0.001)
    
#segm_deblend_arr_f1.insert(err_ind_f1[1], segm_deblend_arr_f0[err_ind_f1[1]])

In [ ]:
segment_arr_f1

In [ ]:
# median background removal is creating issues so need to keep the zero mask from the initial arrays

In [ ]:
masked_image_arr, combined_highz_mask_arr, other_segments_mask_arr = [], [], [] # this code is combining masks of similar redshifts galaxies in the cutouts (z>3.08)

for ind in range(good_ind.size):
    ra_s, dec_s = c0[2].data[cat_indx_match[ind][0]]['ra'], c0[2].data[cat_indx_match[ind][0]]['dec']    
    coordinates_s = SkyCoord(ra=ra_s, dec=dec_s, unit="deg")
    pixel_coords_s = peak_cutout_filt0_arr[ind].wcs.world_to_pixel(coordinates_s)
    phot_zz = c0[-1].data[cat_indx_match[ind][0]]['EAZY_z_a']
    z99ll = z_99l[cat_indx_match[ind][0]]
    z99ul = z_99u[cat_indx_match[ind][0]]
    if len(phot_zz)<1:
        ra_s, dec_s, phot_zz = [sample_data['ra'][ind]], [sample_data['dec'][ind]], [sample_data['zphot'][ind]] 
    if ind == 20:
        dec_s = [dec_s[0] - 1*u.arcsecond.to(u.deg)]
    masked_image, combined_highz_mask, other_segments_mask = mask_highz_galaxies(data=data_sub_arr_f0[ind], segment=segm_deblend_arr_f0[ind], wcs=peak_cutout_filt0_arr[ind].wcs,\
                                                                         galaxy_ra=ra_s, galaxy_dec=dec_s, galaxy_redshifts=phot_zz, z_lim=3.08) # doing for f814 filter here
    
    masked_image_arr       .append(masked_image)  # masked image without target galaxies
    combined_highz_mask_arr.append(combined_highz_mask) # target mask
    other_segments_mask_arr.append(other_segments_mask) # background mask

In [ ]:
masked_image_arr        = np.delete(np.array(masked_image_arr)       , 7, axis=0)
combined_highz_mask_arr = np.delete(np.array(combined_highz_mask_arr), 7, axis=0)
other_segments_mask_arr = np.delete(np.array(other_segments_mask_arr), 7, axis=0)
data_sub_arr_f0         = np.delete(np.array(data_sub_arr_f0)        , 7, axis=0)

In [ ]:
masked_image_arr2, combined_highz_mask_arr2, other_segments_mask_arr2 = [], [], []

for ind in range(7):
    ra_s, dec_s = c0[2].data[cat_indx_match[ind][0]]['ra'], c0[2].data[cat_indx_match[ind][0]]['dec']    
    coordinates_s = SkyCoord(ra=ra_s, dec=dec_s, unit="deg")
    pixel_coords_s = peak_cutout_filt1_arr[ind].wcs.world_to_pixel(coordinates_s)
    phot_zz = c0[-1].data[cat_indx_match[ind][0]]['EAZY_z_a']
    z99ll = z_99l[cat_indx_match[ind][0]]
    z99ul = z_99u[cat_indx_match[ind][0]]
    if len(phot_zz)<1:
        ra_s, dec_s, phot_zz = [sample_data['ra'][ind]], [sample_data['dec'][ind]], [sample_data['zphot'][ind]] 
    if ind == 20:
        dec_s = [dec_s[0] - 1*u.arcsecond.to(u.deg)]
    masked_image, combined_highz_mask, other_segments_mask = mask_highz_galaxies(data=data_sub_arr_f1[ind], segment=segm_deblend_arr_f1[ind], wcs=peak_cutout_filt0_arr[ind].wcs,\
                                                                         galaxy_ra=ra_s, galaxy_dec=dec_s, galaxy_redshifts=phot_zz, z_lim=3.08) # doing for f336 filter here
    
    masked_image_arr2       .append(masked_image)
    combined_highz_mask_arr2.append(combined_highz_mask)
    other_segments_mask_arr2.append(other_segments_mask)

In [ ]:
len(combined_highz_mask_arr), len(data_sub_arr_f0), len(data_sub_arr_f1)

In [ ]:
# saving the target pixels
combined_highz_mask_arr = np.array(combined_highz_mask_arr)
img_filt0_masked_arr = np.ma.masked_array(data_sub_arr_f0, mask=~combined_highz_mask_arr)
img_filt1_masked_arr = np.ma.masked_array(data_sub_arr_f1, mask=~combined_highz_mask_arr)

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.imshow(data_sub_arr_f1[20], origin='lower')
plt.imshow(combined_highz_mask_arr[19], origin='lower', vmin=-0.1, vmax=0.1, alpha=0.3)
plt.subplot(122)
plt.imshow(np.ma.masked_less_equal(data_sub_arr_f1[20], value=0), origin='lower', cmap='viridis', vmin=0, vmax=.1)
plt.imshow(combined_highz_mask_arr[19], origin='lower', vmin=-0.1, vmax=0.1, alpha=0.3)

In [ ]:
import scipy.ndimage as ndimage
#plt.imshow(ndimage.gaussian_filter(data_sub_arr_f1[i], sigma=3))

In [ ]:
# zero mask from the original datasets
zero_mask_f0 = (np.array(img_filt0_arr) == 0)
zero_mask_f1 = (np.array(img_filt1_arr) == 0)

zero_mask_comb = zero_mask_f0 + zero_mask_f1

In [ ]:
# removing all sources including the target galaxies
all_source_mask = combined_highz_mask_arr + other_segments_mask_arr + zero_mask_f0
img_filt0_only_bg_masked_arr = np.copy(data_sub_arr_f0)
img_filt0_only_bg_masked_arr[all_source_mask] = np.nan


In [ ]:
# specially for F336 we need to remove the other sources from the image, so another mask will be added here
all_source_mask_with_f1 = combined_highz_mask_arr + other_segments_mask_arr + other_segments_mask_arr2 + zero_mask_comb
img_filt1_only_bg_masked_arr = np.copy(data_sub_arr_f1)
img_filt1_only_bg_masked_arr[all_source_mask_with_f1] = np.nan

In [ ]:
## getting the backgrond images now

img_filt0_bg_filled_1 = fill_masked_pixels(img_filt0_only_bg_masked_arr)
img_filt1_bg_filled_1 = fill_masked_pixels(img_filt1_only_bg_masked_arr)

# this is first round of bg filling we might need to resegment and do the process again(specially for f336)

In [ ]:
plt.imshow(img_filt0_arr[20], origin='lower', vmin=0.0001, vmax=0.5)
plt.imshow(combined_highz_mask_arr[20] , origin='lower', alpha=0.1)
#plt.colorbar()

In [ ]:
# resegmenting f0, f1

segm_deblend_arr_f0, segment_arr_f0, data_sub_arr_f0, convolved_data_arr_f0,\
    closest_mask_arr_f0, background_filled_arr2_f0, background_filled_arr3_f0, err_ind_f0 = get_segments_background(img_filt0_bg_filled_1, box_size_=4,\
                                                                                                                                filt_size=5,\
                                                                                                                                npixels=5,\
                                                                                                                                nlevels=32,\
                                                                                                                                contrast=0.0001)

In [ ]:
segm_deblend_arr_f1, segment_arr_f1, data_sub_arr_f1, convolved_data_arr_f1,\
    closest_mask_arr_f1, background_filled_arr2_f1, background_filled_arr3_f1, err_ind_f1 = get_segments_background(img_filt1_bg_filled_1, box_size_=4,\
                                                                                                                                filt_size=5,\
                                                                                                                                npixels=5,\
                                                                                                                                nlevels=32,\
                                                                                                                                contrast=0.0001)

In [ ]:
def get_none_ind(data): # since the data already polished some of the images dont get any sources to segement so we need to keep those images, and work on other images
    none_ind = []
    for i in range(len(data)):
        if data[i] == None:
            none_ind.append(i)
            
    none_ind = np.array(none_ind).astype(int)
    
    return none_ind

In [ ]:
none_ind_f0 = np.delete(np.arange(len(segment_arr_f0)), get_none_ind(data=segment_arr_f0))
none_ind_f1 = np.delete(np.arange(len(segment_arr_f0)), get_none_ind(data=segment_arr_f1))

In [ ]:
# now filling with bg only the which are not none
# this time we just need to fill the segment map mask pixels with background rms

len(data_sub_arr_f1), len(data_sub_arr_f0)

In [ ]:
def new_segment(data, none_ind):

    new_seg = []
    for i in range(len(none_ind)):
        new_seg.append(data[none_ind[i]].data)
    new_seg  = np.array(new_seg)
    
    return new_seg

In [ ]:
segment_arr_f0 = new_segment(data=segment_arr_f0, none_ind=none_ind_f0)
segment_arr_f1 = new_segment(data=segment_arr_f1, none_ind=none_ind_f1)

segment_arr_f0 = (segment_arr_f0 >= 1)
segment_arr_f1 = (segment_arr_f1 >= 1)

In [ ]:
none_ind_f0

In [ ]:
full_seg_f0 = np.zeros(shape=np.array(data_sub_arr_f0).shape).astype(bool)
full_seg_f1 = np.zeros(shape=np.array(data_sub_arr_f1).shape).astype(bool)

full_seg_f0[none_ind_f0] = segment_arr_f0
full_seg_f1[none_ind_f1] = segment_arr_f1

In [ ]:
data_sub_arr_f0_2 = np.copy(data_sub_arr_f0)
data_sub_arr_f1_2 = np.copy(data_sub_arr_f1)

#data_sub_arr_f0_2 = np.ma.masked_array(data_sub_arr_f0_2, mask=full_seg_f0)
#data_sub_arr_f1_2 = np.ma.masked_array(data_sub_arr_f1_2, mask=full_seg_f1)

data_sub_arr_f0_2[full_seg_f0] = 0
data_sub_arr_f1_2[full_seg_f1] = 0

In [ ]:
for i in range(len(data_sub_arr_f0_2)):
    plt.imshow(np.ma.masked_equal(data_sub_arr_f1_2[i], value=0))
    plt.show()

In [ ]:
data_sub_arr_f0_2[none_ind_f0]

In [ ]:



img_filt0_bg_filled_2 = fill_masked_pixels(img_filt0_only_bg_masked_arr)
img_filt1_bg_filled_2 = fill_masked_pixels(img_filt1_only_bg_masked_arr)

In [ ]:
for i in range(len(data_sub_arr_f0)):
    plt.imshow(data_sub_arr_f1[i])
    plt.show()

In [ ]:
segm_deblend_arr_f1 = np.array(segm_deblend_arr_f1)

In [ ]:
img_bg_filt0_arr, img_filt0_arr = process_images_with_masks(
    data_arrays=data_sub_arr_f0,
    source_masks=combined_highz_mask_arr,
    background_masks=other_segments_mask_arr,
    good_indices=good_ind,
    fill_masked_function=fill_masked_with_noise2, filt=False)

img_bg_filt1_arr, img_filt1_arr = process_images_with_masks(
    data_arrays=data_sub_arr_f1,
    source_masks=combined_highz_mask_arr,
    background_masks=other_segments_mask_arr,
    good_indices=good_ind,
    fill_masked_function=fill_masked_with_noise2, filt=False)

In [ ]:
# plotting the cutouts
# Define the number of rows and columns for the grid
rows, cols = 6, 5

# Create the figure and axes for the grid
fig, axes = plt.subplots(rows, cols, figsize=(15, 15))

fig.suptitle(r'$f814~with ~source mask$', fontsize=20, fontweight='bold', y=1.01)
# Flatten the axes array to iterate easily
axes = axes.flatten()

# Calculate global min and max for scaling
#all_images = np.array(peak_cutout_wts0_arr)  # Assuming this is the list/array of images


# Plot each image on the grid
for i, ax in enumerate(axes):
    if i < len(img_filt0_arr):
        # Plot the image with consistent scaling
        xpix, ypix = peak_cutout_filt1_arr[i].wcs.world_to_pixel_values(sample_data['ra'][i]*u.deg, sample_data['dec'][i]*u.deg)    
        if i ==20:
            ypix = ypix - 25
        #ax.imshow(img_filt1_arr2[i], origin='lower', cmap='viridis', vmin=-0.1, vmax=0.1) 
        ax.imshow(img_filt1_arr[i], origin='lower', vmin=-0.1, vmax=0.1)
        #ax.plot(xpix, ypix, color='black', marker='x', markersize=10,) 
        ax.set_title(r"$\rm %d$" % i, fontsize=8)

        # Disable ticks
        ax.set_xticks([])  # Disable x-axis ticks
        ax.set_yticks([])  # Disable y-axis ticks
    else:
        # Hide the empty subplots
        ax.axis('off')  # Hide unused slots

# Adjust layout to minimize space between images
plt.tight_layout(pad=-0.1, h_pad=0.01, w_pad=0.01)
#plt.savefig('control_gal_samples_final_f336.jpg', bbox_inches='tight', dpi=220)
# Show the plot
plt.show()

In [ ]:
img_filt0_arr[20] = np.zeros_like(img_filt0_arr[0])
img_filt1_arr[20] = np.zeros_like(img_filt0_arr[0])
wts_filt0_arr[20] = np.zeros_like(img_filt0_arr[0])
wts_filt1_arr[20]

In [ ]:
# for eelgs
mask_arr_                  = get_cleaned_data(arr=combined_highz_mask_arr , arr_ind=good_ind)
img_filt0_arr_             = get_cleaned_data(arr=img_filt0_arr,            arr_ind=good_ind)
img_filt1_arr_             = get_cleaned_data(arr=img_filt1_arr,            arr_ind=good_ind)
img_bg_filt0_arr_          = get_cleaned_data(arr=img_bg_filt0_arr,         arr_ind=good_ind)
img_bg_filt1_arr_          = get_cleaned_data(arr=img_bg_filt1_arr,         arr_ind=good_ind)
wts_filt0_arr_             = get_cleaned_data(arr=wts_filt0_arr,            arr_ind=good_ind)
wts_filt1_arr_             = get_cleaned_data(arr=wts_filt1_arr,            arr_ind=good_ind)
wcs_filt0_arr_             = get_cleaned_data(arr=peak_cutout_filt0_arr,    arr_ind=good_ind)
wcs_filt1_arr_             = get_cleaned_data(arr=peak_cutout_filt1_arr,    arr_ind=good_ind)

In [ ]:
round_=1
np.save('/media/himanshu/MyHDD/data_galaxy_evolution/process_data/mask_arr_'                 + 'eelgs', mask_arr_           ,)
np.save('/media/himanshu/MyHDD/data_galaxy_evolution/process_data/good_index'                + 'eelgs', good_ind            ,)
np.save('/media/himanshu/MyHDD/data_galaxy_evolution/process_data/wcs_filt0_arr_'            + 'eelgs', wcs_filt0_arr_      ,)
np.save('/media/himanshu/MyHDD/data_galaxy_evolution/process_data/wcs_filt1_arr_'            + 'eelgs', wcs_filt1_arr_      ,)
np.save('/media/himanshu/MyHDD/data_galaxy_evolution/process_data/img_bg_filt0_arr_'         + 'eelgs', img_bg_filt0_arr_   ,)
np.save('/media/himanshu/MyHDD/data_galaxy_evolution/process_data/img_bg_filt1_arr_'         + 'eelgs', img_bg_filt1_arr_   ,)
np.save('/media/himanshu/MyHDD/data_galaxy_evolution/process_data/img_filt0_arr_'            + 'eelgs', img_filt0_arr_      ,)
np.save('/media/himanshu/MyHDD/data_galaxy_evolution/process_data/img_filt1_arr_'            + 'eelgs', img_filt1_arr_      ,)
np.save('/media/himanshu/MyHDD/data_galaxy_evolution/process_data/wts_filt0_arr_'            + 'eelgs', wts_filt0_arr_      ,)
np.save('/media/himanshu/MyHDD/data_galaxy_evolution/process_data/wts_filt1_arr_'            + 'eelgs', wts_filt1_arr_      ,)